In [1]:
! pip install pandas_datareader
! pip install yfinance

     |████████████████████████████████| 5.5MB 13.4MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=f58bce0710a364519bc225ba6f208bccd304031913277fd5d0999e53376843a4
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [3]:
#Create csv for data by the day for past 10 years for AAPL
import pandas as pd
import yfinance as y
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn.model_selection import *
from sklearn.decomposition import *
from pandas_datareader import data as pdr

tickers = "AAPL"

In [4]:
def z_score(df):
    # copy the dataframe
    df_std = df.copy()
    # apply the z-score method
    for column in df_std.columns:
        df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()

    return df_std

#Choose arbitrary Tech

#apple = y.Ticker(tickers)

y.pdr_override()
apple = pdr.get_data_yahoo(tickers = tickers, period = "max", interval = "1d", auto_adjust = True, prepost = False)

apple = pd.DataFrame(apple)
print(apple['Open'].count())

apple_train = pd.DataFrame(apple[0:8132])
apple_test = pd.DataFrame(apple[8132:])

apple_train.to_csv(tickers + "_min_train.csv")
apple_test.to_csv(tickers + "_min_test.csv") 

[*********************100%***********************]  1 of 1 completed
10166


In [5]:
data = pd.read_csv('/content/AAPL_min_train.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
print(data.head())
print(data.info())
print(data['Open'].head())
data['Open'] = 1
print(data['Open'].head())
print(data['Open'].count())

scaler = MinMaxScaler(feature_range=(-1, 1)) 
    #amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    #amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

series = scaler.fit_transform(data.to_numpy().reshape(-1, 1)).reshape(-1)
print(series)

                Open      High       Low     Close     Volume
Date                                                         
1980-12-12  0.100922  0.101361  0.100922  0.100922  469033600
1980-12-15  0.096096  0.096096  0.095657  0.095657  175884800
1980-12-16  0.089075  0.089075  0.088636  0.088636  105728000
1980-12-17  0.090830  0.091268  0.090830  0.090830   86441600
1980-12-18  0.093463  0.093902  0.093463  0.093463   73449600
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8132 entries, 1980-12-12 to 2013-03-12
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    8132 non-null   float64
 1   High    8132 non-null   float64
 2   Low     8132 non-null   float64
 3   Close   8132 non-null   float64
 4   Volume  8132 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 381.2 KB
None
Date
1980-12-12    0.100922
1980-12-15    0.096096
1980-12-16    0.089075
1980-12-17    0.090830
1980-12-18    0.093463
Name: 

In [7]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot

torch.manual_seed(0)
np.random.seed(0)

# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) # (S,N,E) 
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)

input_window = 200 # number of input steps
output_window = 1 # number of prediction steps, in this model its fixed to one
batch_size = 10 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]
          

class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

def get_data():
       
    from sklearn.preprocessing import MinMaxScaler
    
    #loading series data from a file
    #from pandas import read_csv
    #series = pd.read_csv('/content/AAPL.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    
    #Choose arbitrary tech
    y.pdr_override()
    apple = pdr.get_data_yahoo(tickers = tickers, period = "max", interval = "1d", auto_adjust = True, prepost = False)

    apple = pd.DataFrame(apple)
    #print(apple['Open'].count())

    apple_train = pd.DataFrame(apple[0:8132])
    apple_test = pd.DataFrame(apple[8132:])


    train_open = apple_train['Open'].to_numpy()
    train_high = apple_train['High'].to_numpy()
    train_low = apple_train['Low'].to_numpy()
    train_volume = apple_train['Volume'].to_numpy()
    train_close = apple_train['Close'].to_numpy()
    
    test_open = apple_test['Open'].to_numpy()
    test_high = apple_test['High'].to_numpy()
    test_low = apple_test['Low'].to_numpy()
    test_volume = apple_test['Volume'].to_numpy()
    test_close = apple_test['Close'].to_numpy()

    # looks like normalizing input values critical for the model
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    #amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    #amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

    apple_train = scaler.fit_transform(apple_train.to_numpy().reshape(-1, 1)).reshape(-1)
    train_close = scaler.fit_transform(train_close.reshape(-1, 1)).reshape(-1)
    train_high = scaler.fit_transform(train_high.reshape(-1, 1)).reshape(-1)
    train_low = scaler.fit_transform(train_low.reshape(-1, 1)).reshape(-1)
    train_volume = scaler.fit_transform(train_volume.reshape(-1, 1)).reshape(-1)
    #close = scaler.fit_transform(full_data['Close'].reshape(-1, 1)).reshape(-1)

    apple_test = scaler.fit_transform(apple_test.to_numpy().reshape(-1, 1)).reshape(-1)
    test_close = scaler.fit_transform(test_close.reshape(-1, 1)).reshape(-1)
    test_high = scaler.fit_transform(test_high.reshape(-1, 1)).reshape(-1)
    test_low = scaler.fit_transform(test_low.reshape(-1, 1)).reshape(-1)
    test_volume = scaler.fit_transform(test_volume.reshape(-1, 1)).reshape(-1)

    #split into train_data and test_data
    #samples = 2516
    #train_data = close[:int(samples*0.8)]
    #test_data = close[int(samples*0.8):]

    samples = 8132
    train_data = train_close
    test_data = test_close

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment.. 
    train_sequence = create_inout_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window] #todo: fix hack? -> din't think this through, looks like the last n sequences are to short, so I just remove them. Hackety Hack.. 

    #test_data = torch.FloatTensor(test_data).view(-1) 
    test_data = create_inout_sequences(test_data,input_window)
    test_data = test_data[:-output_window] #todo: fix hack?

    return train_sequence.to(device),test_data.to(device)

def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target


def train(train_data):
    model.train() # Turn on the train mode \o/
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            output = eval_model(data)            
            total_loss += criterion(output, target).item()
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0)
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy() -> no need to detach stuff.. 
    len(test_result)

    pyplot.plot(test_result,color="red", label = 'Test result')
    pyplot.plot(truth[:500],color="blue", label = 'Actual Close')
    #pyplot.plot(test_result-truth,color="green", label = 'Test result - Actual Close')
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.legend(loc = 'upper left')
    pyplot.savefig('transformer-epoch%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i


# predict the next n steps based on the input data 
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps):            
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    
    # I used this plot to visualize if the model picks up any long term structure within the data. 
    pyplot.plot(data,color="red", label = 'Predictions for next steps')       
    pyplot.plot(data[:input_window],color="blue", label = 'Close values')    
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.legend(loc = 'upper left')
    pyplot.savefig('transformer-future%d.png'%steps)
    pyplot.close()
        

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            total_loss += len(data[0])* criterion(output, targets).cpu().item()
    return total_loss / len(data_source)

train_data, val_data = get_data()
model = TransAm().to(device)

criterion = nn.MSELoss()
lr = 0.005 
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

best_val_loss = float("inf")
epochs = 200 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    if(epoch % 10 is 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data, 200)
    else:
        val_loss = evaluate(model, val_data)
   
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step() 

#src = torch.rand(input_window, batch_size, 1) # (source sequence length,batch size,feature number) 
#out = model(src)
#
#print(out)
#print(out.shape)


[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |   158/  793 batches | lr 0.005000 | 10.19 ms | loss 1.74774 | ppl     5.74
| epoch   1 |   316/  793 batches | lr 0.005000 |  9.70 ms | loss 0.00231 | ppl     1.00
| epoch   1 |   474/  793 batches | lr 0.005000 |  9.65 ms | loss 0.00024 | ppl     1.00
| epoch   1 |   632/  793 batches | lr 0.005000 |  9.71 ms | loss 0.00063 | ppl     1.00
| epoch   1 |   790/  793 batches | lr 0.005000 |  9.72 ms | loss 0.01907 | ppl     1.02
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  8.32s | valid loss 1.14763 | valid ppl     3.15
-----------------------------------------------------------------------------------------
| epoch   2 |   158/  793 batches | lr 0.004513 |  9.86 ms | loss 0.03342 | ppl     1.03
| epoch   2 |   316/  793 batches | lr 0.004513 |  9.84 ms | loss 0.00027 | ppl     1.00
| epoch   2 |   474/  793 batches | lr 0.004513 |  9.82 ms | loss 0.00020 | ppl     1.00
| epoch   2 |   632/  793 batche

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch  11 |   158/  793 batches | lr 0.002844 |  9.57 ms | loss 0.00722 | ppl     1.01
| epoch  11 |   316/  793 batches | lr 0.002844 |  9.58 ms | loss 0.00014 | ppl     1.00
| epoch  11 |   474/  793 batches | lr 0.002844 |  9.60 ms | loss 0.00013 | ppl     1.00
| epoch  11 |   632/  793 batches | lr 0.002844 |  9.53 ms | loss 0.00034 | ppl     1.00
| epoch  11 |   790/  793 batches | lr 0.002844 |  9.63 ms | loss 0.00356 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  11 | time:  8.13s | valid loss 0.11228 | valid ppl     1.12
-----------------------------------------------------------------------------------------
| epoch  12 |   158/  793 batches | lr 0.002702 |  9.70 ms | loss 0.00560 | ppl     1.01
| epoch  12 |   316/  793 batches | lr 0.002702 |  9.62 ms | loss 0.00014 | ppl     1.00
| epoch  12 |   474/  793 batches | lr 0.002702 |  9.61 ms | loss 0.00012 | ppl     1.00
| epoch  12 |   632/  793 batche

...

| epoch 171 |   158/  793 batches | lr 0.000001 |  9.61 ms | loss 0.00004 | ppl     1.00
| epoch 171 |   316/  793 batches | lr 0.000001 |  9.61 ms | loss 0.00003 | ppl     1.00
| epoch 171 |   474/  793 batches | lr 0.000001 |  9.62 ms | loss 0.00004 | ppl     1.00
| epoch 171 |   632/  793 batches | lr 0.000001 |  9.61 ms | loss 0.00007 | ppl     1.00
| epoch 171 |   790/  793 batches | lr 0.000001 |  9.66 ms | loss 0.00071 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch 171 | time:  8.15s | valid loss 0.00150 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch 172 |   158/  793 batches | lr 0.000001 |  9.58 ms | loss 0.00004 | ppl     1.00
| epoch 172 |   316/  793 batches | lr 0.000001 |  9.57 ms | loss 0.00003 | ppl     1.00
| epoch 172 |   474/  793 batches | lr 0.000001 |  9.55 ms | loss 0.00004 | ppl     1.00
| epoch 172 |   632/  793 batche

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch 181 |   158/  793 batches | lr 0.000000 |  9.63 ms | loss 0.00004 | ppl     1.00
| epoch 181 |   316/  793 batches | lr 0.000000 |  9.62 ms | loss 0.00003 | ppl     1.00
| epoch 181 |   474/  793 batches | lr 0.000000 |  9.61 ms | loss 0.00004 | ppl     1.00
| epoch 181 |   632/  793 batches | lr 0.000000 |  9.59 ms | loss 0.00007 | ppl     1.00
| epoch 181 |   790/  793 batches | lr 0.000000 |  9.65 ms | loss 0.00071 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch 181 | time:  8.16s | valid loss 0.00151 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch 182 |   158/  793 batches | lr 0.000000 |  9.60 ms | loss 0.00004 | ppl     1.00
| epoch 182 |   316/  793 batches | lr 0.000000 |  9.60 ms | loss 0.00003 | ppl     1.00
| epoch 182 |   474/  793 batches | lr 0.000000 |  9.56 ms | loss 0.00004 | ppl     1.00
| epoch 182 |   632/  793 batche

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch 191 |   158/  793 batches | lr 0.000000 |  9.61 ms | loss 0.00004 | ppl     1.00
| epoch 191 |   316/  793 batches | lr 0.000000 |  9.64 ms | loss 0.00003 | ppl     1.00
| epoch 191 |   474/  793 batches | lr 0.000000 |  9.63 ms | loss 0.00004 | ppl     1.00
| epoch 191 |   632/  793 batches | lr 0.000000 |  9.71 ms | loss 0.00007 | ppl     1.00
| epoch 191 |   790/  793 batches | lr 0.000000 |  9.68 ms | loss 0.00072 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch 191 | time:  8.18s | valid loss 0.00150 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch 192 |   158/  793 batches | lr 0.000000 |  9.69 ms | loss 0.00004 | ppl     1.00
| epoch 192 |   316/  793 batches | lr 0.000000 |  9.62 ms | loss 0.00003 | ppl     1.00
| epoch 192 |   474/  793 batches | lr 0.000000 |  9.66 ms | loss 0.00004 | ppl     1.00
| epoch 192 |   632/  793 batche